In [3]:
import pandas as pd
import xarray as xr

from scipy import optimize as opt
from scipy import stats

In [12]:
ds = xr.open_dataset( "../results/prec_hist_hist_idf_valores.nc" )
ds

<xarray.Dataset>
Dimensions:         (south_north: 112, west_east: 132, DURACION: 22, TIEMPO_RETORNO: 8)
Coordinates:
  * south_north     (south_north) int64 0 1 2 3 4 5 ... 106 107 108 109 110 111
  * west_east       (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * DURACION        (DURACION) int64 1 2 3 4 5 6 8 10 ... 48 60 72 84 96 108 120
  * TIEMPO_RETORNO  (TIEMPO_RETORNO) float64 5.0 10.0 25.0 ... 200.0 500.0 1e+03
    LONGITUD        (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
    LATITUD         (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
Data variables:
    INTENSIDAD      (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...

In [19]:
ds = xr.open_dataset( "../results/prec_hist_hist_tretorno.nc" )
ds

<xarray.Dataset>
Dimensions:         (south_north: 112, west_east: 132, DURACION: 22, TIEMPO_RETORNO: 31)
Coordinates:
  * south_north     (south_north) int64 0 1 2 3 4 5 ... 106 107 108 109 110 111
  * west_east       (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * DURACION        (DURACION) int64 1 2 3 4 5 6 8 10 ... 48 60 72 84 96 108 120
  * TIEMPO_RETORNO  (TIEMPO_RETORNO) float64 1.032 1.067 1.103 ... 16.0 32.0
    LONGITUD        (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
    LATITUD         (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
Data variables:
    AÑO             (south_north, west_east, DURACION, TIEMPO_RETORNO) int64 ...
    INTENSIDAD      (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
    PROBABILIDAD    (south_north, west_east, DURACION, TIEMPO_RETORNO) float64 ...

In [4]:
path = "../results/prec_hist_hist_tretorno.nc"

df = xr.open_dataset( path ).to_dataframe()

# Quitamos la intensidad, año, probabilidad, tiempo de retorno, y agregamos
# columnas para los parámetros de la distribución.
df_2 = df.copy().drop( ["PROBABILIDAD", "AÑO"], axis = 1  #)
    ).reset_index( "TIEMPO_RETORNO" )
#df_2 = df_2.reset_index( "DURACION" )
df_3 = df_2.copy(
    #).reset_index( "TIEMPO_RETORNO"
    ).drop( ["INTENSIDAD", "TIEMPO_RETORNO"], axis = 1).groupby(
    ["south_north", "west_east", "DURACION"] ).mean()
cols = ["GEV_C", "GEV_LOC", "GEV_SCALE", "KTEST_P"]
df_3[ cols ] = None
t = ( ( [5, 10, 25, 50, 100, 200, 500, 1000] + [None] * 23 )
    * df_2.index.get_level_values("west_east").unique().shape[0]
    * df_2.index.get_level_values("south_north").unique().shape[0]
    * df_2.index.get_level_values("DURACION").unique().shape[0] )
#df_2["DURACION"] = d
df_2["TIEMPO_RETORNO"] = t
df_2 = df_2.dropna().set_index( "TIEMPO_RETORNO", append = True )

df_2

KeyError: "['PROBABILIDAD' 'AÑO'] not found in axis"

In [12]:


# Iteramos para cada celda y duración.
#for i in df_3.index.get_level_values("south_north").unique():
for i in range(0, 3):
    print(f"Calculando coordenada y #{i}...")
    # for j in df_3.index.get_level_values("west_east").unique():
    for j in range(0, 3):
        for k in df_3.index.get_level_values("DURACION").unique():
            # ajustamos la distribución de valores extremos.
            params = stats.genextreme.fit( df.loc[ (i, j, k), "INTENSIDAD" ] )
            # Hacemos la prueba Kolmogorov Smirnoff.
            pvalue = stats.kstest( df.loc[ (i, j, k), "INTENSIDAD" ],
                stats.genextreme(*params).cdf ).pvalue
            df_3.loc[ (i, j, k), cols ] = [*params] + [pvalue]
            # Calculamos las intensidades de acuerdo con la distribución.
            df_2.loc[ (i, j, k), "INTENSIDAD"] = stats.genextreme(
                *df_3.loc[ (i, j, k), cols[:-1] ] ).isf(
                1 / df_2.loc[ (i, j, k) ].index.get_level_values(
                "TIEMPO_RETORNO") )

Calculando coordenada y #0...


/opt/homebrew/Caskroom/miniforge/base/envs/gv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:406: RuntimeWarning: invalid value encountered in true_divide
  return m3 / np.power(m2, 1.5)


Calculando coordenada y #1...
Calculando coordenada y #2...


In [19]:
df_2.to_xarray().set_coords( ["LONGITUD", "LATITUD"] ).drop_vars(["south_north", "west_east"])

<xarray.Dataset>
Dimensions:         (DURACION: 22, TIEMPO_RETORNO: 8, south_north: 112, west_east: 132)
Coordinates:
  * DURACION        (DURACION) int64 1 2 3 4 5 6 8 10 ... 48 60 72 84 96 108 120
  * TIEMPO_RETORNO  (TIEMPO_RETORNO) float64 5.0 10.0 25.0 ... 200.0 500.0 1e+03
    LONGITUD        (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
    LATITUD         (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
Dimensions without coordinates: south_north, west_east
Data variables:
    INTENSIDAD      (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...

In [116]:
path = "../temp/*.nc"

def pre(ds):
    ds = ( ds.expand_dims( ESTADISTICA = int( 
        #ds["XLAT"].encoding["source"][-4:-3] )
        1 )
    
        ).rename_dims( { "XTIME":"MES" } 
        ).rename_vars( { "XLAT": "LATITUD", "XLONG": "LONGITUD",
        "XTIME": "MES", "Pcp": "PRECIPITACION" }
        ).drop_vars( ["XTIME_bnds"] )
        )
    return ds

ds = xr.open_mfdataset( path,
            combine = "nested", 
            concat_dim = "ESTADISTICA",
            parallel = True, preprocess = pre
            )

ds["MES"] = range(1, 13)
ds["ESTADISTICA"] = range(1, 5)
#ds = ds.swap_dims( ["south_north", "west_east", "MES", "ESTADISTICA"] )

ds.to_dataframe().reorder_levels( ["south_north",
    "west_east", "MES", "ESTADISTICA"] ).sort_index(
    ).to_xarray().set_coords( ["LONGITUD", "LATITUD"] )

<xarray.Dataset>
Dimensions:        (south_north: 112, west_east: 132, MES: 12, ESTADISTICA: 4)
Coordinates:
  * south_north    (south_north) int64 0 1 2 3 4 5 6 ... 106 107 108 109 110 111
  * west_east      (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * MES            (MES) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * ESTADISTICA    (ESTADISTICA) int64 1 2 3 4
    LONGITUD       (south_north, west_east, MES, ESTADISTICA) float32 -100.5 ...
    LATITUD        (south_north, west_east, MES, ESTADISTICA) float32 18.14 ....
Data variables:
    PRECIPITACION  (south_north, west_east, MES, ESTADISTICA) float32 0.0 ......